
# Imports

In [18]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import re
from selenium.common.exceptions import NoSuchElementException
import math
import datetime

### 1. Chrome
### 2. Login no SISREGII 
### 3. Agenda do SISREGII 

In [19]:
# 1
navegador = webdriver.Chrome()
navegador.get("https://sisregiii.saude.gov.br/")
# 2
navegador.find_element(By.XPATH,    '//*[@id="usuario"]').send_keys("120191056SAMUEL")
navegador.find_element(By.XPATH,'//*[@id="senha"]').send_keys("147258")
navegador.find_element(By.XPATH,'//*[@id="conteudoFull"]/div[1]/div[1]/div[8]/input').click()
# 3
navegador.get("https://sisregiii.saude.gov.br/cgi-bin/cons_agendas")

#### Filtro de dias
OBS.: O restante dos filtros serão obtidos a partir do HTML do SISREGII

In [20]:
data_atual = datetime.date.today()
dia_atual = data_atual.day
mes_atual = data_atual.month
dia_str = str(dia_atual)
mes_str = str(mes_atual).zfill(2)
data_inicial = "02/09/2024"
data_final = "25/09/2024"

navegador.find_element(By.ID,'dataInicial').send_keys(data_inicial)
navegador.find_element(By.ID,'dataFinal').send_keys(data_final)

### Filtros
##### Pega pelo HTML quantas unidades estão disponíveis no sistema.
- (1) Unidades de saúde;
- (2) Profissionais;
- (3) Procedimentos.
##### Quando os dados são obtidos, então buscamos em cada unidade de saúde, cada profissional disponível e qual procediemento realiza. Feito isso pegamos de uma tabela os seguintes dados:
- Vagas de 1ª Vez Solicitadas;
- Solicitaçoes Confirmadas;
- Solicitações Pendentes;
- Solicitações com falta;
- Total de solicitações.

In [21]:
todas_linhas = []

# 1 - Coleta a quantidade de unidades
select_element = navegador.find_element(By.NAME, "ups")
options = select_element.find_elements(By.TAG_NAME, "option")
num_options_unidades = len(options)


# Seleciona uma unidade por vez
for i in range(2, num_options_unidades + 1):
    try: 
        navegador.find_element(By.XPATH, f'//*[@id="main_page"]/form/center/table/tbody/tr[5]/td[2]/select/option[{i}]').click()

        # 2 - Coleta a quantidade de profissionais
        select_element = navegador.find_element(By.NAME, "cpf")
        options = select_element.find_elements(By.TAG_NAME, "option")
        num_options_profissionais = len(options)

        # Seleciona um profissional por vez
        for j in range(2, num_options_profissionais + 1):
            try:
                navegador.find_element(By.XPATH, f'//*[@id="main_page"]/form/center/table/tbody/tr[6]/td[2]/select/option[{j}]').click()

                # 3 - Coleta a quantidade de procedimentos
                select_element = navegador.find_element(By.NAME, "pa")
                options = select_element.find_elements(By.TAG_NAME, "option")
                num_options_procedimentos = len(options)
                
                # Seleciona um procedimento por vez
                for k in range(2, num_options_procedimentos + 1):
                    try:
                        navegador.find_element(By.XPATH, f'//*[@id="main_page"]/form/center/table/tbody/tr[7]/td[2]/select/option[{k}]').click()

                        # Clica no botão de consulta e depois em OK
                        navegador.find_element(By.XPATH, '//*[@id="main_page"]/form/center/table/tbody/tr[8]/td[2]/select/option').click()
                        navegador.find_element(By.XPATH, '//*[@id="main_page"]/form/center/table/tbody/tr[12]/td/input').click()  
                        
                        try:
                            vagas_solicitadas = navegador.find_element(By.XPATH, '//*[@id="main_page"]/form/center[3]/span/b').text
                        except:
                            # Encontre todas as tabelas na página 
                            # 1ª tabela: Dados do procedimento
                            tables = navegador.find_elements(By.CLASS_NAME, 'table_listagem')
                            last_table = tables[-1]
                            # Encontre todas as linhas na tabela
                            rows_last_table = last_table.find_elements(By.TAG_NAME, 'tr')
                            # Crie uma lista para armazenar os dados dos valores
                            data = []
                            # Crie uma lista para armazenar as descrições (cabeçalhos das colunas)
                            headers = []
                            # Itere sobre as linhas da tabela, ignorando a primeira que é o cabeçalho
                            for row in rows_last_table[1:]:
                                cols = row.find_elements(By.TAG_NAME, 'td')
                                # Verifique se há exatamente 2 colunas, uma com o texto (descrição) e outra com o valor
                                if len(cols) == 2:
                                    description = cols[0].text.strip()  # O texto é a descrição (cabeçalho da coluna)
                                    value = cols[1].text.strip()  # O valor é o dado correspondente
                                    # Adicione a descrição à lista de cabeçalhos, se ainda não estiver presente
                                    if description not in headers:
                                        headers.append(description)
                                    # Adicione o valor à lista de dados
                                    data.append(value)
                            # Criar um DataFrame do Pandas com uma única linha
                            df = pd.DataFrame([data], columns=headers)  
                            df['Unidade de Saúde Solicitante'] = navegador.find_element(By.XPATH, '//*[@id="main_page"]/form/center[2]/table/tbody/tr[2]/td[2]').text
                            df['Profissional'] = navegador.find_element(By.XPATH, '//*[@id="main_page"]/form/center[2]/table/tbody/tr[4]/td[2]').text
                            df['Procedimento'] = navegador.find_element(By.XPATH, '//*[@id="main_page"]/form/center[2]/table/tbody/tr[5]/td[2]').text
                            todas_linhas.append(df)
                    except:
                        k = k-1
                        navegador.get("https://sisregiii.saude.gov.br/cgi-bin/cons_agendas")
                        navegador.find_element(By.ID,'dataInicial').send_keys(data_inicial)
                        navegador.find_element(By.ID,'dataFinal').send_keys(data_final)
                        navegador.find_element(By.XPATH, f'//*[@id="main_page"]/form/center/table/tbody/tr[5]/td[2]/select/option[{i}]').click()
                        navegador.find_element(By.XPATH, f'//*[@id="main_page"]/form/center/table/tbody/tr[6]/td[2]/select/option[{j}]').click()
                            
            except:
                j = j-1
                navegador.get("https://sisregiii.saude.gov.br/cgi-bin/cons_agendas")
                navegador.find_element(By.ID,'dataInicial').send_keys(data_inicial)
                navegador.find_element(By.ID,'dataFinal').send_keys(data_final)
                navegador.find_element(By.XPATH, f'//*[@id="main_page"]/form/center/table/tbody/tr[5]/td[2]/select/option[{i}]').click()

    except:
        i = i-1
        navegador.get("https://sisregiii.saude.gov.br/cgi-bin/cons_agendas")
        navegador.find_element(By.ID,'dataInicial').send_keys(data_inicial)
        navegador.find_element(By.ID,'dataFinal').send_keys(data_final)

df_final = pd.concat(todas_linhas, ignore_index=True)
df_final["data_inicial"] = pd.to_datetime(data_inicial, format='%d/%m/%Y')
df_final["data_final"] = pd.to_datetime(data_final, format='%d/%m/%Y')


#### Subindo pro banco

In [22]:
connection = pymysql.connect(
    host='databasedevdb.integrator.host',
    user='samuel',
    password='Esl!FoNdZbv7ziDQ',
    db='tabelas_regulacao',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

try:
    print(df_final)
    with connection.cursor() as cursor:
        for index, row in df_final.iterrows():
            insert_sql = '''
            INSERT INTO tabela_agenda 
            (vagas_primeira_vez_solicitadas, 
             vagas_retorno_solicitadas, 
             vagas_primeira_vez_consumidas, 
             vagas_retorno_consumidas, 
             vagas_reserva_consumidas, 
             solicitacoes_confirmadas, 
             solicitacoes_pendentes, 
             solicitacoes_com_falta, 
             total_solicitacoes, 
             unidade_saude_solicitante, 
             profissional, 
             procedimento,
             data_inicial,
             data_final)
            VALUES 
            (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
            '''
            cursor.execute(insert_sql, (
            row['Vagas de 1ª Vez Solicitadas:'], 
            row['Vagas de Retorno Solicitadas:'], 
            row['Vagas de 1ª Vez Consumidas:'], 
            row['Vagas de Retorno Consumidas:'], 
            row['Vagas de Reserva Consumidas:'], 
            row['Solicitaçoes Confirmadas:'], 
            row['Solicitações Pendentes:'], 
            row['Solicitações com Falta:'], 
            row['Total de solicitações:'], 
            row['Unidade de Saúde Solicitante'], 
            row['Profissional'], 
            row['Procedimento'],
            row['data_inicial'],
            row['data_final']
        ))
        
        # Confirme a transação
        connection.commit()

finally:
    # Fechar a conexão
    connection.close()

    Vagas de 1ª Vez Solicitadas: Vagas de Retorno Solicitadas:  \
0                              1                             0   
1                              7                             0   
2                              3                             0   
3                              1                             0   
4                              2                             0   
..                           ...                           ...   
156                          118                             0   
157                          113                             0   
158                            5                             0   
159                          105                             0   
160                            3                             0   

    Vagas de 1ª Vez Consumidas: Vagas de Retorno Consumidas:  \
0                             1                            0   
1                             7                            0   
2              